# extract feats

In [4]:
import os
import sys
sys.path.append('../')

import numpy as np
import torch


# import vox_model_bank
from train import train_model_new
from train.read_data import *
from train.my_dataloader import *
# from sklearn.metrics import roc_curve


In [5]:
model_path = '/Lun2/rzz/kaldi-master/egs/zhiyong/sre19/exp/Standard_ETDNN_plus/ckpt/min_eer.model'
model_id = 'Standard_ETDNN_plus'
model_metric = 'AM_normfree_softmax_anneal_ce_head'

In [8]:
component_dir = '../../sdsvc_exp/backend_exp/Standard_ETDNN_plus_c'
PLDA_DIM = 370
PLDA_DATA_NAME = 'plda_data'
TEST_DATA_NAME = 'test_data_370'
PLDA_PARA_NAME = 'plda_para_370'
SCORING_PLDA_NAME = 'score_plda_370'
SCORING_COSINE_NAME = 'score_cosine'
ENR_DATA_NAME = 'enr_data'
EVL_DATA_NAME = 'evl_data'
TRAIN_DATA_NAME = 'train_data'

In [9]:
if not os.path.isdir(component_dir):
    os.makedirs(component_dir)

# GPU multiprocess for train

In [10]:
from multiprocessing import Process, Manager

In [11]:
num_p = 10

In [45]:
# train_list = '/Lun0/zhiyong/dataset/plda_full_data.csv'
train_list = '/Lun0/zhiyong/SdSV_2020_deepmine/train_mfcc.csv'
# train_data = CSVDataSet(train_list)
train_data = PickleDataSet(train_list)

In [9]:
train_data_len = len(train_data)
num_per_process = (85764 // num_p) + 1

In [10]:
manager = Manager()
class_list_new_m = manager.list()

In [11]:
data_m = []
# class_list_new_m = []
for i in range(num_p):
    data = torch.utils.data.Subset(train_data, np.arange(i*num_per_process, min((i+1)*num_per_process, train_data_len)))
    data_m.append(data)
    class_list_new_m.append({})
    print(len(data))

8577
8577
8577
8577
8577
8577
8577
8577
8577
8571


In [12]:
def extract_feature_m(i, train_data, class_list_new_m):
#     train_list = '/Lun0/zhiyong/dataset/vox12_kaldi_train_data/vox12_kaldi_train_data.csv'
    model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

    torch.backends.cudnn.benchmark = False
    if i < 26:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(i%2)
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#     torch.cuda.set_device(i%2)
    device = torch.device("cuda:0")

#     train_data = CSVDataSet(train_list)
#     train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=False)

    train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
    batch_sampler=None, num_workers=8, collate_fn=None,\
    pin_memory=False, drop_last=False, timeout=0,\
    worker_init_fn=None, multiprocessing_context=None)

    model = train_model_new.get_model(model_id, model_metric, None, model_settings, None)
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)
    model = model.to(device)
    model.eval()

    class_list_new = {}

    for count, (batch_x, batch_y) in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        batch_x = batch_x.to(device)
        label = batch_y[0].split('-')[0]
        batch_y = torch.tensor([0]).to(device)
        try:
            with torch.no_grad():
                _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')
        except:
            print('Proc', str(i), 'EER:', label)
            continue
    #     _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

        emb = emb.squeeze().data.cpu().numpy()

        if label not in class_list_new.keys():
            class_list_new[label] = emb[None, :]
        else:
            class_list_new[label] = np.append(class_list_new[label], emb[None, :], axis=0)

        if (count+1) % 10000 == 0:
            print('Proc '+ str(i) + ':' + str((count+1) // 10000))
    
    class_list_new_m[i] = class_list_new
    del model, batch_x, batch_y
    torch.cuda.empty_cache()

In [ ]:
processes = [Process(target = extract_feature_m, args = (i, data_m[i], class_list_new_m)) for i in range(num_p)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

In [14]:
count = 0
for i in class_list_new_m:
    for j in i:
        count += len(i[j])
count

0

In [15]:
class_list_new = class_list_new_m[0]

for count, this_list in enumerate(class_list_new_m):
    if count == 0:
        continue
    for this_label in this_list:
        if this_label not in class_list_new.keys():
            class_list_new[this_label] = this_list[this_label]
        else:
            class_list_new[this_label] = np.append(class_list_new[this_label], this_list[this_label], axis=0)

In [16]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])
count

0

In [175]:
nanlist = []
for i in class_list_new:
    if np.isnan(class_list_new[i]).any():
        print(i)
        nanlist.append(i)
for i in nanlist:
    class_list_new.pop(i)

In [176]:
count

85764

In [177]:
len(class_list_new)

588

In [178]:
import pickle
out = component_dir+'/'+TRAIN_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(class_list_new, handle)

# GPU multiprocess for enr

In [12]:
from multiprocessing import Process, Manager

In [13]:
num_p = 10

In [14]:
train_list = '/Lun0/zhiyong/SdSV_2020_deepmine/enr_mfcc.csv'
train_data = PickleDataSet(train_list)

In [16]:
train_data_len

110673

In [15]:
train_data_len = len(train_data)
num_per_process = (110673 // num_p) + 1

In [17]:
manager = Manager()
class_list_new_m = manager.list()

In [18]:
data_m = []
# class_list_new_m = []
for i in range(num_p):
    data = torch.utils.data.Subset(train_data, np.arange(i*num_per_process, min((i+1)*num_per_process, train_data_len)))
    data_m.append(data)
    class_list_new_m.append({})
    print(len(data))

11068
11068
11068
11068
11068
11068
11068
11068
11068
11061


In [19]:
def extract_feature_m(i, train_data, class_list_new_m):
#     train_list = '/Lun0/zhiyong/dataset/vox12_kaldi_train_data/vox12_kaldi_train_data.csv'
    model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 10249, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

    torch.backends.cudnn.benchmark = False
    if i < 26:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(i%2)
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#     torch.cuda.set_device(i%2)
    device = torch.device("cuda:0")

#     train_data = CSVDataSet(train_list)
#     train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=False)

    train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
    batch_sampler=None, num_workers=8, collate_fn=None,\
    pin_memory=False, drop_last=False, timeout=0,\
    worker_init_fn=None, multiprocessing_context=None)

    model = train_model_new.get_model(model_id, model_metric, None, model_settings, None)
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)
    model = model.to(device)
    model.eval()

    class_list_new = {}

    for count, (batch_x, batch_y) in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        batch_x = batch_x.to(device)
        label = batch_y[0].split('-')[0]
        batch_y = torch.tensor([0]).to(device)
        try:
            with torch.no_grad():
                _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')
        except:
            print('Proc', str(i), 'EER:', label)
            continue
    #     _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

        emb = emb.squeeze().data.cpu().numpy()

        if label not in class_list_new.keys():
            class_list_new[label] = emb[None, :]
        else:
            class_list_new[label] = np.append(class_list_new[label], emb[None, :], axis=0)

        if (count+1) % 10000 == 0:
            print('Proc '+ str(i) + ':' + str((count+1) // 10000))
    
    class_list_new_m[i] = class_list_new
    del model, batch_x, batch_y
    torch.cuda.empty_cache()

In [20]:
processes = [Process(target = extract_feature_m, args = (i, data_m[i], class_list_new_m)) for i in range(num_p)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Proc 3:1
Proc 8:1
Proc 6:1
Proc 4:1
Proc 5:1
Proc 1:1
Proc 9:1
Proc 7:1
Proc 0:1
Proc 2:1


In [21]:
count = 0
for i in class_list_new_m:
    for j in i:
        count += len(i[j])
count

110673

In [22]:
class_list_new = class_list_new_m[0]

for count, this_list in enumerate(class_list_new_m):
    if count == 0:
        continue
    for this_label in this_list:
        if this_label not in class_list_new.keys():
            class_list_new[this_label] = this_list[this_label]
        else:
            class_list_new[this_label] = np.append(class_list_new[this_label], this_list[this_label], axis=0)

In [23]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])
count

110673

In [24]:
nanlist = []
for i in class_list_new:
    if np.isnan(class_list_new[i]).any():
        print(i)
        nanlist.append(i)
for i in nanlist:
    class_list_new.pop(i)

In [25]:
count

110673

In [26]:
len(class_list_new)

110673

In [27]:
import pickle
out = component_dir+'/'+ENR_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(class_list_new, handle)

# GPU multiprocess for evl

In [28]:
from multiprocessing import Process, Manager

In [29]:
num_p = 10

In [30]:
# train_list = '/Lun0/zhiyong/dataset/plda_full_data.csv'
train_list = '/Lun0/zhiyong/SdSV_2020_deepmine/evl_mfcc.csv'
# train_data = CSVDataSet(train_list)
train_data = PickleDataSet(train_list)

In [31]:
train_data_len = len(train_data)
num_per_process = (69542 // num_p) + 1

In [32]:
manager = Manager()
class_list_new_m = manager.list()

In [33]:
data_m = []
# class_list_new_m = []
for i in range(num_p):
    data = torch.utils.data.Subset(train_data, np.arange(i*num_per_process, min((i+1)*num_per_process, train_data_len)))
    data_m.append(data)
    class_list_new_m.append({})
    print(len(data))

6955
6955
6955
6955
6955
6955
6955
6955
6955
6947


In [34]:
def extract_feature_m(i, train_data, class_list_new_m):
#     train_list = '/Lun0/zhiyong/dataset/vox12_kaldi_train_data/vox12_kaldi_train_data.csv'
    model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 10249, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

    torch.backends.cudnn.benchmark = False
    if i < 26:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(i%2)
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#     torch.cuda.set_device(i%2)
    device = torch.device("cuda:0")

#     train_data = CSVDataSet(train_list)
#     train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=False)

    train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
    batch_sampler=None, num_workers=8, collate_fn=None,\
    pin_memory=False, drop_last=False, timeout=0,\
    worker_init_fn=None, multiprocessing_context=None)

    model = train_model_new.get_model(model_id, model_metric, None, model_settings, None)
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)
    model = model.to(device)
    model.eval()

    class_list_new = {}

    for count, (batch_x, batch_y) in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        batch_x = batch_x.to(device)
        label = batch_y[0].split('-')[0]
        batch_y = torch.tensor([0]).to(device)
        try:
            with torch.no_grad():
                _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')
        except:
            print('Proc', str(i), 'EER:', label)
            continue
    #     _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

        emb = emb.squeeze().data.cpu().numpy()

        if label not in class_list_new.keys():
            class_list_new[label] = emb[None, :]
        else:
            class_list_new[label] = np.append(class_list_new[label], emb[None, :], axis=0)

        if (count+1) % 10000 == 0:
            print('Proc '+ str(i) + ':' + str((count+1) // 10000))
    
    class_list_new_m[i] = class_list_new
    del model, batch_x, batch_y
    torch.cuda.empty_cache()

In [35]:
processes = [Process(target = extract_feature_m, args = (i, data_m[i], class_list_new_m)) for i in range(num_p)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank


In [36]:
count = 0
for i in class_list_new_m:
    for j in i:
        count += len(i[j])
count

69542

In [37]:
class_list_new = class_list_new_m[0]

for count, this_list in enumerate(class_list_new_m):
    if count == 0:
        continue
    for this_label in this_list:
        if this_label not in class_list_new.keys():
            class_list_new[this_label] = this_list[this_label]
        else:
            class_list_new[this_label] = np.append(class_list_new[this_label], this_list[this_label], axis=0)

In [38]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])
count

69542

In [39]:
nanlist = []
for i in class_list_new:
    if np.isnan(class_list_new[i]).any():
        print(i)
        nanlist.append(i)
for i in nanlist:
    class_list_new.pop(i)

In [40]:
count

69542

In [41]:
len(class_list_new)

69542

In [42]:
import pickle
out = component_dir+'/'+EVL_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(class_list_new, handle)

# GPU multiprocess for plda

In [5]:
from multiprocessing import Process, Manager

In [6]:
num_p = 28

In [7]:
train_list = '/Lun0/zhiyong/dataset/plda_full_data.csv'
# train_list = '/Lun0/zhiyong/dataset/plda_full_data_noVAD.csv'
# train_data = CSVDataSet(train_list)
train_data = PickleDataSet(train_list)

In [8]:
train_data_len = len(train_data)
num_per_process = (1276888 // num_p) + 1

In [9]:
manager = Manager()
class_list_new_m = manager.list()

In [10]:
data_m = []
# class_list_new_m = []
for i in range(num_p):
    data = torch.utils.data.Subset(train_data, np.arange(i*num_per_process, min((i+1)*num_per_process, train_data_len)))
    data_m.append(data)
    class_list_new_m.append({})
    print(len(data))

45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45580


In [11]:
def extract_feature_m(i, train_data, class_list_new_m):
#     train_list = '/Lun0/zhiyong/dataset/vox12_kaldi_train_data/vox12_kaldi_train_data.csv'
    model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

    torch.backends.cudnn.benchmark = False
    if i < 26:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(i%2)
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#     torch.cuda.set_device(i%2)
    device = torch.device("cuda:0")

#     train_data = CSVDataSet(train_list)
#     train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=False)

    train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
    batch_sampler=None, num_workers=8, collate_fn=None,\
    pin_memory=False, drop_last=False, timeout=0,\
    worker_init_fn=None, multiprocessing_context=None)

    model = train_model_new.get_model(model_id, model_metric, None, model_settings, None)
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)
    model = model.to(device)
    model.eval()

    class_list_new = {}

    for count, (batch_x, batch_y) in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        batch_x = batch_x.to(device)
        label = batch_y[0].split('-')[0]
        batch_y = torch.tensor([0]).to(device)
        try:
            with torch.no_grad():
                _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')
        except:
            print('Proc', str(i), 'EER:', label)
            continue
    #     _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

        emb = emb.squeeze().data.cpu().numpy()

        if label not in class_list_new.keys():
            class_list_new[label] = emb[None, :]
        else:
            class_list_new[label] = np.append(class_list_new[label], emb[None, :], axis=0)

        if (count+1) % 10000 == 0:
            print('Proc '+ str(i) + ':' + str((count+1) // 10000))
    
    class_list_new_m[i] = class_list_new
    del model, batch_x, batch_y
    torch.cuda.empty_cache()

In [12]:
processes = [Process(target = extract_feature_m, args = (i, data_m[i], class_list_new_m)) for i in range(num_p)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Using new training model bank
Proc 27:1
Proc 24:1
Proc 26:1
Proc 22:1
Proc 20:1
Proc 16:1
Proc 10:1
Proc 6:1
Proc 8:1
Proc 18:1
Proc 4:1
Proc 25:1
Proc 2:1
Proc 12:1
Proc 14:1
Proc 21:1
Proc

In [13]:
count = 0
for i in class_list_new_m:
    for j in i:
        count += len(i[j])
count

1276888

In [14]:
class_list_new = class_list_new_m[0]

for count, this_list in enumerate(class_list_new_m):
    if count == 0:
        continue
    for this_label in this_list:
        if this_label not in class_list_new.keys():
            class_list_new[this_label] = this_list[this_label]
        else:
            class_list_new[this_label] = np.append(class_list_new[this_label], this_list[this_label], axis=0)

In [17]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])
count

1276888

In [18]:
nanlist = []
for i in class_list_new:
    if np.isnan(class_list_new[i]).any():
        print(i)
        nanlist.append(i)
for i in nanlist:
    class_list_new.pop(i)

In [19]:
count

1276888

In [20]:
len(class_list_new)

7323

In [21]:
import pickle
out = component_dir+'/'+PLDA_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(class_list_new, handle)

In [20]:
# class_list_new = {}
# for i in class_list:
#     class_list_new[i[:-1]] = class_list[i]

# PLDA_FIN

In [21]:
import os
import sys
sys.path.append('./train')

import numpy as np
import torch


# import vox_model_bank
from train import train_model_new
from train.read_data import *
from train.my_dataloader import *
# from sklearn.metrics import roc_curve

In [86]:
from kaldi_plda import *
from kaldi_lda import *
import numpy as np

In [129]:
import pickle
with open('/Lun2/rzz/kaldi-master/egs/zhiyong/sre19/new_testbench'+'/'+PLDA_DATA_NAME, 'rb') as handle:
    class_list_new = pickle.load(handle)

In [130]:
# Substract global mean
global_mean = np.zeros(512)
num_utt = 0
for count, i in enumerate(class_list_new):
    num_utt += class_list_new[i].shape[0]
    global_mean += class_list_new[i].shape[0] * np.mean(class_list_new[i], axis=0)
    
global_mean = (1.0 / num_utt) * global_mean
print('Norm of mean:', np.linalg.norm(global_mean))

Norm of mean: 1.9705881360946418


In [131]:
for i in class_list_new:
    class_list_new[i] = class_list_new[i] - global_mean

In [26]:
# # normlize to sqrt(dim)
# for i in class_list_new:
#     scale = np.sqrt(512) / np.linalg.norm(class_list_new[i], axis=1, keepdims=True)
#     class_list_new[i] = scale * class_list_new[i]

In [132]:
lda = LDA(lda_dim=PLDA_DIM, ivector_dim=512)
for i in class_list_new:
    lda.AccStats(class_list_new[i])
print('lda norm of global mean:', lda.GetGlobalMean()[1])

lda norm of global mean: 9.641873606430675e-08


In [133]:
transform = lda.ComputeLdaTransform()

the input data has norm of mean 9.641873606430675e-08
[7.01190490e+00 6.75681662e+00 6.53853053e+00 6.25568993e+00
 6.11248365e+00 5.95853316e+00 5.76067349e+00 5.64741497e+00
 5.57487744e+00 5.41990231e+00 5.38111534e+00 5.31902969e+00
 5.23789065e+00 5.19969617e+00 5.15591191e+00 5.06367837e+00
 4.95002558e+00 4.87409618e+00 4.84840474e+00 4.74871468e+00
 4.71036534e+00 4.62632340e+00 4.59444607e+00 4.53859824e+00
 4.50825004e+00 4.44792223e+00 4.42513736e+00 4.34977219e+00
 4.28159505e+00 4.25209140e+00 4.19223498e+00 4.16862494e+00
 4.06981441e+00 4.06355494e+00 4.03968798e+00 3.96515890e+00
 3.92289551e+00 3.89805072e+00 3.86241858e+00 3.79790583e+00
 3.72304290e+00 3.67162826e+00 3.64237800e+00 3.57782648e+00
 3.55594916e+00 3.51338202e+00 3.48159699e+00 3.44671244e+00
 3.38799819e+00 3.37222889e+00 3.31932983e+00 3.26479589e+00
 3.23583721e+00 3.18890552e+00 3.16877421e+00 3.13774669e+00
 3.09139334e+00 3.04371133e+00 3.00723616e+00 2.97468080e+00
 2.93475707e+00 2.91202466e+00 

In [29]:
# # normlize to sqrt(dim)
# for i in class_list_new:
#     scale = np.sqrt(512) / np.linalg.norm(class_list_new[i], axis=1, keepdims=True)
#     class_list_new[i] = scale * class_list_new[i]

In [134]:
for i in class_list_new:
    class_list_new[i] = class_list_new[i].dot(transform.T)

In [135]:
# normlize to sqrt(dim)
for i in class_list_new:
    scale = np.sqrt(PLDA_DIM) / np.linalg.norm(class_list_new[i], axis=1, keepdims=True)
    class_list_new[i] = scale * class_list_new[i]

In [136]:
plda_stats = PldaStats(PLDA_DIM)
for i in class_list_new:
    plda_stats.add_samples(class_list_new[i])

In [137]:
plda_stats.sort()
plda_stats.is_sorted()

True

In [138]:
# test_fin_prior
plda_estimator = PldaEstimation(plda_stats)
plda_paras = plda_estimator.estimate(iteration=5)

1 5
nllr_x: 411.3536651606943
nllr_y: -371.394730264205
normalized_nllr: 39.95893489648931
nllr_m: 412.13621538628524
nllr_m_2: 412.13621538628604
part1_residual -455.29228631886446
part2_mean -412.13621538628524
normlized_obj -455.04478465275184
2 5
nllr_x: 240.15276805056112
nllr_y: -458.94205476522495
normalized_nllr: -218.7892867146638
nllr_m: 246.5813414941023
nllr_m_2: 246.5813414941021
part1_residual -437.3886274503415
part2_mean -246.5813414941023
normlized_obj -436.294340594284
3 5
nllr_x: 237.46553834505684
nllr_y: -458.8741951366892
normalized_nllr: -221.40865679163232
nllr_m: 245.48345787472775
nllr_m_2: 245.48345787472786
part1_residual -437.3910096499305
part2_mean -245.48345787472775
normlized_obj -436.2904127286267
4 5
nllr_x: 237.20065986248494
nllr_y: -458.7576755590343
normalized_nllr: -221.55701569654934
nllr_m: 245.46150954196784
nllr_m_2: 245.4615095419679
part1_residual -437.3910074823672
part2_mean -245.46150954196784
normlized_obj -436.2902846989926
5 5
nllr_x:

In [139]:
import pickle
out = component_dir+'/'+ PLDA_PARA_NAME
with open(out, 'wb') as handle:
    pickle.dump(plda_paras, handle)

# Load PLDA model

In [140]:
import pickle
with open(component_dir+'/'+ PLDA_PARA_NAME, 'rb') as handle:
    plda_paras = pickle.load(handle)

In [141]:
plda = PLDA(plda_paras[0], plda_paras[1], plda_paras[2])

# Unsupervised adaptation

In [179]:
import pickle
with open(component_dir+'/'+TRAIN_DATA_NAME, 'rb') as handle:
    train_data = pickle.load(handle)

In [183]:
unlabeled_list = np.zeros([0, 512])
for i in train_data:
    unlabeled_list = np.append(unlabeled_list, train_data[i], axis=0)

In [185]:
major_mean = np.mean(unlabeled_list, axis=0)

In [31]:
# import pickle
# out = component_dir+'/major_mean'
# with open(out, 'wb') as handle:
#     pickle.dump(major_mean, handle)

In [186]:
unlabeled_list = unlabeled_list - major_mean

In [187]:
unlabeled_list = unlabeled_list.dot(transform.T)

In [188]:
# normlize to sqrt(dim)
scale = np.sqrt(PLDA_DIM) / np.linalg.norm(unlabeled_list, axis=1, keepdims=True)
unlabeled_list = scale * unlabeled_list

In [190]:
adaptor = PldaUnsupervisedAdaptor(dim=PLDA_DIM)

In [191]:
for i in range(unlabeled_list.shape[0]):
    adaptor.add_stats(unlabeled_list[i])

In [192]:
new_plda_paras = adaptor.update_plda(plda_paras[0], plda_paras[1], plda_paras[2])

In [193]:
import pickle
out = component_dir+'/new_plda_paras'
with open(out, 'wb') as handle:
    pickle.dump(new_plda_paras, handle)

In [194]:
plda = PLDA(new_plda_paras[0], new_plda_paras[1], new_plda_paras[2])

# Enroll models

In [43]:
import pickle
with open(component_dir+'/'+ENR_DATA_NAME, 'rb') as handle:
    enr_data = pickle.load(handle)

In [44]:
enr_list = {}
num_utt = {}
with open('/Lun0/zhiyong/SdSV_2020_deepmine/task2_enrollment/docs/model_enrollment.txt', 'r') as f:
    for count, line in enumerate(f):
        if count == 0:
            continue
        info = line[:-1].split(' ')
        model_label = info[0]
        num_utt[model_label] = len(info)-1
        for i in range(1, len(info)):
            if model_label not in enr_list.keys():
                enr_list[model_label] = enr_data[info[i]]
            else:
                enr_list[model_label] = np.append(enr_list[model_label], enr_data[info[i]], axis=0)

# Test data

In [45]:
import pickle
with open(component_dir+'/'+EVL_DATA_NAME, 'rb') as handle:
    evl_data = pickle.load(handle)

# Scoring PLDA

In [43]:
trail_path = '/Lun0/zhiyong/SdSV_2020_deepmine/task2_enrollment/docs/trials.txt'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [ ]:
for i in enr_list:
    enr_list[i] = np.mean(enr_list[i], axis=0).squeeze()
    enr_list[i] = enr_list[i] - global_mean
    enr_list[i] = transform.dot(enr_list[i])
    enr_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(enr_list[i])) * enr_list[i]
    num = num_utt[i]
    enr_list[i] = plda.transform_ivector(enr_list[i], num)

In [ ]:
for i in evl_data:
    evl_data[i] = evl_data[i].squeeze()
    evl_data[i] = evl_data[i] - global_mean
    evl_data[i] = transform.dot(evl_data[i])
    evl_data[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(evl_data[i])) * evl_data[i]
    evl_data[i] = plda.transform_ivector(evl_data[i], 1)

In [ ]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            line = line[:-1]
            if count == 0:
                print(line)
                continue
            enroll_emb = enr_list[line.split(' ')[0]].squeeze()
            num = num_utt[line.split(' ')[0]]
            test_emb = evl_data[line.split(' ')[1]].squeeze()

            cosine = plda.log_likelihood_ratio(enroll_emb, num, test_emb)
            
            of.write(str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

In [156]:
count

13198024

# Scoring adapt plda

In [198]:
trail_path = '/Lun0/zhiyong/SdSV_2020_deepmine/task2_enrollment/docs/trials.txt'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME+'adapt'

In [199]:
for i in enr_list:
    enr_list[i] = np.mean(enr_list[i], axis=0).squeeze()
    enr_list[i] = enr_list[i] - major_mean
    enr_list[i] = transform.dot(enr_list[i])
    enr_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(enr_list[i])) * enr_list[i]
    num = num_utt[i]
    enr_list[i] = plda.transform_ivector(enr_list[i], num)

In [200]:
for i in evl_data:
    evl_data[i] = evl_data[i].squeeze()
    evl_data[i] = evl_data[i] - major_mean
    evl_data[i] = transform.dot(evl_data[i])
    evl_data[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(evl_data[i])) * evl_data[i]
    evl_data[i] = plda.transform_ivector(evl_data[i], 1)

In [201]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            line = line[:-1]
            if count == 0:
                print(line)
                continue
            enroll_emb = enr_list[line.split(' ')[0]].squeeze()
            num = num_utt[line.split(' ')[0]]
            test_emb = evl_data[line.split(' ')[1]].squeeze()

            cosine = plda.log_likelihood_ratio(enroll_emb, num, test_emb)
            
            of.write(str(cosine)+'\n')
            
            if (count+1) % 100000 == 0:
                print((count+1) // 100000)

model-id evaluation-file-id
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0


In [48]:
count

13198024

# Cosine Scoring

In [46]:
trail_path = '/Lun0/zhiyong/SdSV_2020_deepmine/task2_enrollment/docs/trials.txt'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [47]:
for i in enr_list:
    enr_list[i] = np.mean(enr_list[i], axis=0).squeeze()
    enr_list[i] = (1.0 / np.linalg.norm(enr_list[i])) * enr_list[i]

In [48]:
for i in evl_data:
    evl_data[i] = evl_data[i].squeeze()
    evl_data[i] = (1.0 / np.linalg.norm(evl_data[i])) * evl_data[i]

In [49]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            line = line[:-1]
            if count == 0:
                print(line)
                continue
            enroll_emb = enr_list[line.split(' ')[0]].squeeze()
            test_emb = evl_data[line.split(' ')[1]].squeeze()

            cosine = np.dot(enroll_emb, test_emb)
            
            of.write('{:.3f}'.format(cosine)+'\n')
            
            if (count+1) % 100000 == 0:
                print((count+1) // 100000)

model-id evaluation-file-id
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
